<a href="https://colab.research.google.com/github/zakiyarofiq/Project_Akhir/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torchsummary as summary
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import random_split
import torch.nn.functional as F
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
import time
import copy
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
data_dir = './gdrive/MyDrive/datatraining/'
classes = os.listdir(data_dir)
classes

['eksim', 'psoriasis', 'scabies']

In [ ]:
def get_transforms(mode='train'):
    transform = []
    transform.append(T.Resize((512, 512)))
    transform.append(T.ToTensor())
    transform.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transform)

In [ ]:
dataset = ImageFolder(data_dir, transform=get_transforms())
dataset

Dataset ImageFolder
    Number of datapoints: 140
    Root location: ./gdrive/MyDrive/datatraining/
    StandardTransform
Transform: Compose(
               Resize(size=(512, 512), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               RandomHorizontalFlip(p=0.5)
           )

In [ ]:
len(dataset)

140

In [ ]:
img, label = dataset[3]
print(img.shape, label)
img

torch.Size([3, 512, 512]) 0


tensor([[[0.8588, 0.8784, 0.8902,  ..., 0.9647, 0.9882, 0.9961],
         [0.8706, 0.8784, 0.8902,  ..., 0.9725, 0.9922, 0.9882],
         [0.8824, 0.8784, 0.8863,  ..., 0.9843, 0.9961, 0.9804],
         ...,
         [0.6353, 0.6510, 0.6510,  ..., 0.7098, 0.7216, 0.7176],
         [0.6275, 0.6471, 0.6431,  ..., 0.7098, 0.7216, 0.7176],
         [0.6275, 0.6431, 0.6392,  ..., 0.7098, 0.7216, 0.7176]],

        [[0.5922, 0.6157, 0.6314,  ..., 0.6980, 0.7137, 0.7098],
         [0.6039, 0.6157, 0.6314,  ..., 0.7020, 0.7137, 0.6980],
         [0.6157, 0.6118, 0.6235,  ..., 0.7137, 0.7098, 0.6784],
         ...,
         [0.4353, 0.4471, 0.4353,  ..., 0.4314, 0.4471, 0.4510],
         [0.4275, 0.4392, 0.4314,  ..., 0.4314, 0.4471, 0.4510],
         [0.4275, 0.4353, 0.4275,  ..., 0.4314, 0.4471, 0.4510]],

        [[0.5490, 0.5725, 0.5843,  ..., 0.6706, 0.6824, 0.6627],
         [0.5608, 0.5686, 0.5843,  ..., 0.6784, 0.6824, 0.6510],
         [0.5725, 0.5686, 0.5804,  ..., 0.6902, 0.6784, 0.

In [ ]:
random_seed = 42
torch.manual_seed(random_seed);
val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

(112, 28)

In [ ]:
efficientnet_b0 = models.efficientnet_b0(pretrained=True)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

In [ ]:
efficientnet_b0

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [ ]:
model_parameters = filter(lambda p: p.requires_grad, efficientnet_b0.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

5288548

In [ ]:
out = efficientnet_b0(torch.randn((1,3,512,512)))
len(out[0])

1000

In [ ]:
out

tensor([[-2.6817e-01, -5.2940e-02, -3.2159e-01, -1.1753e-01, -2.7741e-01,
          5.5948e-01, -2.7985e-01, -1.1448e-01,  2.2973e-02,  1.7279e-02,
         -1.1561e-01,  2.5404e-02, -1.2456e-01, -1.1565e-01, -3.0263e-01,
         -3.4982e-02, -2.2549e-01, -1.5535e-01, -1.3263e-01, -1.2926e-01,
         -5.2805e-01, -3.3250e-01, -4.4504e-01, -4.7403e-01, -2.3714e-01,
         -5.2113e-01, -5.0976e-01,  2.3945e-03, -1.5497e-01, -7.2515e-01,
         -3.2337e-01, -6.1508e-02,  1.7888e-01, -1.2920e-01, -2.8481e-01,
         -1.5853e-01,  2.3592e-01,  3.6787e-02, -1.8083e-01,  9.3670e-02,
         -1.4057e-01,  3.7923e-02, -5.6863e-01,  1.2553e-02, -2.7906e-01,
          1.5777e-01, -4.4018e-01, -1.7310e-02, -6.6380e-01, -4.4203e-01,
          6.3062e-03, -1.4210e-01, -4.3451e-02,  5.0408e-01,  8.5376e-02,
         -1.5452e-01,  4.5468e-03,  8.8243e-02,  4.6852e-02, -6.0971e-02,
          2.2499e-01, -3.4098e-02, -4.5061e-02,  1.7493e-01, -1.5686e-01,
         -2.4661e-01,  2.3011e-01,  3.

In [ ]:
for param in efficientnet_b0.features[:7].parameters():
    param.requires_grad = False

model_parameters = filter(lambda p: p.requires_grad, efficientnet_b0.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

2410392

In [ ]:
from torch.utils.data.dataloader import DataLoader

batch_size=32
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=2, pin_memory=True)
dataloaders = {'train':train_dl, 'val':val_dl}
dataset_sizes = {'train': len(train_ds), 'val': len(val_ds)}
dataset_sizes

{'train': 112, 'val': 28}

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
model = efficientnet_b0.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [ ]:
# Fn for the training loop
def train_model(model, optimizer, alpha=1, beta=1, num_epochs=10):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print("-"*10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            if phase == 'val':
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for imgs, labels in dataloaders[phase]:
                imgs = imgs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(imgs)
                    _, preds = torch.max(outputs, 1)
                    loss = F.cross_entropy(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * imgs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            writer.add_scalar("Loss/train", epoch_loss, epoch)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model = train_model(model, optimizer, num_epochs=10)

Epoch 1/10
----------
train Loss: 6.8770 Acc: 0.0089
val Loss: 7.5204 Acc: 0.0000

Epoch 2/10
----------
train Loss: 4.2421 Acc: 0.4464
val Loss: 6.5714 Acc: 0.1071

Epoch 3/10
----------
train Loss: 1.7102 Acc: 0.7589
val Loss: 4.6290 Acc: 0.4286

Epoch 4/10
----------
train Loss: 0.8218 Acc: 0.8304
val Loss: 2.4994 Acc: 0.7143

Epoch 5/10
----------
train Loss: 0.4670 Acc: 0.8750
val Loss: 1.2620 Acc: 0.8929

Epoch 6/10
----------
train Loss: 0.2999 Acc: 0.9196
val Loss: 0.6543 Acc: 0.8929

Epoch 7/10
----------
train Loss: 0.1386 Acc: 0.9643
val Loss: 0.3960 Acc: 0.8929

Epoch 8/10
----------
train Loss: 0.0940 Acc: 0.9732
val Loss: 0.3242 Acc: 0.8929

Epoch 9/10
----------
train Loss: 0.0541 Acc: 0.9821
val Loss: 0.2363 Acc: 0.9286

Epoch 10/10
----------
train Loss: 0.0679 Acc: 0.9732
val Loss: 0.1829 Acc: 0.9286

Training complete in 16m 5s
Best val Acc: 0.928571


In [ ]:
torch.save(model, "./skin-model.pt")